# Introduction

This is a helper function to load, hourly intraday or day-ahead spanish electricity price.
It return a pandas dataframe with columns price and timestamps in seconds.

In [1]:
# modules
import pandas as pd
import numpy as np
import datetime

from bs4 import BeautifulSoup
import re
import os
import glob
import requests
from urllib.request import urlopen

In [2]:
def data_loader():
    '''
    download electricity price data from omie data with the following params
    price: day_ahead or intraday_price)
    days: number of days of desired prices, starting from recent available data.
    '''
    omie_link = 'https://www.omie.es/'
    
    price = input('Please type intraday or day-ahead: ')
    days = int(input('Please specify how many days: '))
                     
    if price == 'day-ahead':
        weblink = "https://www.omie.es/en/file-access-list?parents%5B0%5D=/&parents%5B1%5D=Day-ahead%20Market&parents%5B2%5D=1.%20Prices&dir=%20Day-ahead%20market%20hourly%20prices%20in%20Spain&realdir=marginalpdbc"
        col_list = [0,1,2,3,4] 
        skip = 1
        
    elif price == 'intraday':
        weblink = "https://www.omie.es/en/file-access-list?parents%5B0%5D=/&parents%5B1%5D=Continuous%20Intraday%20Market&parents%5B2%5D=1.%20Prices&dir=Maximum%2C%20minimum%20and%20weighted%20price%20for%20each%20hour%20of%20the%20continuous%20intraday%20market&realdir=precios_pibcic"
        col_list = [0,1,2,3,10] 
        skip = 3

    working_dir = os.getcwd()
    local_dir = os.mkdir(os.path.join(working_dir, price))
    os.chdir(price)
    
    #loading files list
    files_list = []
    soup = BeautifulSoup(urlopen(weblink))

    for link in soup.findAll('a'):
        if link.get('href').endswith('.1') or link.get('href').endswith('.2'):
            files_list.append(link.get('href'))
            
    #download files
    files_list = files_list[:days]
    for i in range(len(files_list)):
        r = requests.get(omie_link+files_list[i])
        with open(files_list[i][-10:-2]+'.txt', 'wb') as f: 
            f.write(r.content)
        
    #format to pandas dataframe
    data = pd.DataFrame(columns=['timestamp', price])
    for file in glob.glob('*.txt'):
        try:
          partial_data = pd.read_csv(file,
                                    sep=';',
                                    header=None,
                                    usecols = col_list,
                                    names = ['year', 'month', 'day', 'hour', price],
                                    skiprows=skip,
                                    skipfooter=1,
                                    dtype={'year':int, 'month':int, 'day':int, 'hour':int},
                                    encoding='latin-1',
                                    engine='python'
                                    )
          partial_data['hour'] = partial_data['hour'].replace(24, 0)
          partial_data.drop(partial_data[partial_data.hour > 23].index, inplace=True)

          for i in range(len(partial_data)):
            partial_data.loc[i, 'date'] = str(partial_data.loc[i, 'year'])+'/'+str(partial_data.loc[i, 'month'])+'/'+str(partial_data.loc[i, 'day'])+':'+str(partial_data.loc[i, 'hour'])
            s = str(partial_data.loc[i, 'date'])
            partial_data.loc[i, 'timestamp'] = datetime.datetime.strptime(s,"%Y/%m/%d:%H").timestamp()

          data = pd.concat([data, partial_data[['timestamp', price]]], axis=0).sort_values(by=['timestamp'])
          del partial_data
        except:
          continue
     
    os.chdir(working_dir)
    return data

In [3]:
data = data_loader()

Please type intraday or day-ahead:  day-ahead
Please specify how many days:  5


/tmp/ipykernel_33/788264625.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, partial_data[['timestamp', price]]], axis=0).sort_values(by=['timestamp'])


In [4]:
data

,timestamp,day-ahead
23,1.709856e+09,0.43
0,1.709860e+09,3.20
1,1.709863e+09,0.59
2,1.709867e+09,0.35
3,1.709870e+09,0.05
...,...,...
18,1.710270e+09,60.02
19,1.710274e+09,98.99
20,1.710277e+09,173.82
21,1.710281e+09,117.90


Please upvote if you find it useful.
Happy Kaggling!